In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


2.0.0


In [17]:
# https://www.tensorflow.org/tutorials/keras/regression
years = [
    '200001', '200102', '200203', '200304', '200405', '200506', '200607', '200708', '200809', '200910',
    '201011', '201112', '201213','201314', '201415','201516', '201617', '201718', '201819'
]
for year in years: 
    path = './data/formatted/playoff/playoffdata' + year + '.csv'
    raw_dataset = pd.read_csv(path)
    dataset = raw_dataset.copy()

    # Check for empty row
    print('YEAR: ', year)
    print(dataset.isna().sum())


# Plot data using seaborn
# sns.pairplot(dataset[["winnerSeed", "loserSeed"]], diag_kind="kde")


YEAR:  200001
winnerSeed            0
ID                    0
OverallWin            0
OverallLoss           0
SimpleRanking         0
ScheduleStrength      0
ConferenceWin         0
ConferenceLoss        0
HomeWin               0
HomeLoss              0
AwayWin               0
AwayLoss              0
Points                0
PointsAgainst         0
FieldGoals            0
FieldGoalsAtt         0
3P                    0
3PAtt                 0
FreeThrows            0
FreeThrowsAtt         0
OffensiveReb          0
TotalReb              0
Assists               0
Steals                0
Blocks                0
TurnoverPct           0
PersonalFouls         0
loserSeed             0
ID.1                  0
OverallWin.1          0
OverallLoss.1         0
SimpleRanking.1       0
ScheduleStrength.1    0
ConferenceWin.1       0
ConferenceLoss.1      0
HomeWin.1             0
HomeLoss.1            0
AwayWin.1             0
AwayLoss.1            0
Points.1              0
PointsAgainst.1       0
Fi

YEAR:  201617
winnerSeed            0
ID                    0
OverallWin            0
OverallLoss           0
SimpleRanking         0
ScheduleStrength      0
ConferenceWin         0
ConferenceLoss        0
HomeWin               0
HomeLoss              0
AwayWin               0
AwayLoss              0
Points                0
PointsAgainst         0
FieldGoals            0
FieldGoalsAtt         0
3P                    0
3PAtt                 0
FreeThrows            0
FreeThrowsAtt         0
OffensiveReb          0
TotalReb              0
Assists               0
Steals                0
Blocks                0
TurnoverPct           0
PersonalFouls         0
loserSeed             0
ID.1                  0
OverallWin.1          0
OverallLoss.1         0
SimpleRanking.1       0
ScheduleStrength.1    0
ConferenceWin.1       0
ConferenceLoss.1      0
HomeWin.1             0
HomeLoss.1            0
AwayWin.1             0
AwayLoss.1            0
Points.1              0
PointsAgainst.1       0
Fi